<a href="https://colab.research.google.com/github/2303a51420/reinforcement/blob/main/lab%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "authorship_tag": "ABX9TyOymC+jmASj24GAG1/9MUXC",
      "include_colab_link": True
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/Goutham345/Reinforcement_Learning/blob/main/lab-5.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 3,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "i2KehKKYBT4R",
        "outputId": "b3003792-69de-46d0-f108-a1f8add4ea70"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Using backend: gymnasium\n",
            "Training FrozenLake-v1 | episodes=20000, alpha=0.8, gamma=0.95\n",
            "[  2000/20000] avg_return(last 200) = 0.035, epsilon=0.368\n",
            "[  4000/20000] avg_return(last 200) = 0.145, epsilon=0.135\n",
            "[  6000/20000] avg_return(last 200) = 0.205, epsilon=0.050\n",
            "[  8000/20000] avg_return(last 200) = 0.400, epsilon=0.018\n",
            "[ 10000/20000] avg_return(last 200) = 0.440, epsilon=0.010\n",
            "[ 12000/20000] avg_return(last 200) = 0.425, epsilon=0.010\n",
            "[ 14000/20000] avg_return(last 200) = 0.495, epsilon=0.010\n",
            "[ 16000/20000] avg_return(last 200) = 0.500, epsilon=0.010\n",
            "[ 18000/20000] avg_return(last 200) = 0.540, epsilon=0.010\n",
            "[ 20000/20000] avg_return(last 200) = 0.375, epsilon=0.010\n",
            "\n",
            "Evaluation (greedy policy):\n",
            "Average reward over 100 episodes: 0.240\n",
            "Average steps to termination: 16.1\n",
            "\n",
            "Tips for FrozenLake:\n",
            "- Slippery=True → Use >=20k episodes and slower epsilon decay (0.9997).\n",
            "- Slippery=False → 3k–10k episodes often suffice.\n",
            "\n",
            "Saved: q_table.npy, returns.npy, epsilons.npy\n"
          ]
        }
      ],
      "source": [
        "import argparse\n",
        "import sys\n",
        "import math\n",
        "import numpy as np\n",
        "import random\n",
        "\n",
        "# Try Gymnasium first; fallback to Gym\n",
        "try:\n",
        "    import gymnasium as gym\n",
        "    GYMN = \"gymnasium\"\n",
        "except Exception:\n",
        "    import gym\n",
        "    GYMN = \"gym\"\n",
        "\n",
        "\n",
        "def make_env(env_id: str,\n",
        "             is_slippery: bool | None = None,\n",
        "             render_mode: str | None = None,\n",
        "             seed: int | None = 42):\n",
        "    \"\"\"Create an environment with sensible defaults for FrozenLake and Taxi.\"\"\"\n",
        "    kwargs = {}\n",
        "    if render_mode is not None:\n",
        "        kwargs[\"render_mode\"] = render_mode\n",
        "    if env_id.startswith(\"FrozenLake\") and is_slippery is not None:\n",
        "        kwargs[\"is_slippery\"] = bool(is_slippery)\n",
        "\n",
        "    env = gym.make(env_id, **kwargs)\n",
        "\n",
        "    try:\n",
        "        env.reset(seed=seed)\n",
        "    except TypeError:\n",
        "        pass\n",
        "    if hasattr(env, \"action_space\") and hasattr(env.action_space.seed):\n",
        "        env.action_space.seed(seed)\n",
        "    if hasattr(env, \"observation_space\") and hasattr(env.observation_space.n, \"seed\"):\n",
        "        env.observation_space.seed(seed)\n",
        "    return env\n",
        "\n",
        "\n",
        "def toint(x) -> int:\n",
        "    return int(x) if not isinstance(x, (tuple, list, np.ndarray)) else int(x[0])\n",
        "\n",
        "\n",
        "def greedyaction(q_row: np.ndarray) -> int:\n",
        "    max_val = np.max(q_row)\n",
        "    best_acts = np.flatnonzero(q_row == max_val)\n",
        "    return int(np.random.choice(best_acts))\n",
        "\n",
        "\n",
        "def epsilon_greedy_action(q_table: np.ndarray, state: int, epsilon: float, n_actions: int) -> int:\n",
        "    if random.random() < epsilon:\n",
        "        return random.randrange(n_actions)\n",
        "    return greedyaction(q_table[state])\n",
        "\n",
        "\n",
        "def stepunpack(step_out):\n",
        "    \"\"\"Unpack step results for Gymnasium (5-tuple) and Gym (4-tuple).\"\"\"\n",
        "    if isinstance(step_out, tuple):\n",
        "        if len(step_out) == 5:\n",
        "            next_state, reward, terminated, truncated, info = step_out\n",
        "            done = bool(terminated) or bool(truncated)\n",
        "            return next_state, reward, done\n",
        "        elif len(step_out) == 4:\n",
        "            next_state, reward, done, info = step_out\n",
        "            return next_state, reward, bool(done)\n",
        "    raise RuntimeError(\"Unexpected env.step(...) return format.\")\n",
        "\n",
        "\n",
        "def resetobs(reset_out):\n",
        "    return reset_out[0] if isinstance(reset_out, tuple) else reset_out\n",
        "\n",
        "\n",
        "def train_q_learning(env_id: str = \"FrozenLake-v1\",\n",
        "                     episodes: int = 20000,\n",
        "                     max_steps: int = 200,\n",
        "                     alpha: float = 0.8,\n",
        "                     gamma: float = 0.95,\n",
        "                     epsilon: float = 1.0,\n",
        "                     epsilon_min: float = 0.01,\n",
        "                     epsilon_decay: float = 0.9995,\n",
        "                     is_slippery: bool | None = None,\n",
        "                     seed: int = 42,\n",
        "                     verbose: bool = True):\n",
        "    env = make_env(env_id, is_slippery=is_slippery, seed=seed)\n",
        "    assert hasattr(env.observation_space, 'n') and hasattr(env.action_space, 'n'), \\\n",
        "        \"This Q-learning implementation expects discrete state and action spaces.\"\n",
        "\n",
        "    n_states = int(env.observation_space.n)\n",
        "    n_actions = int(env.action_space.n)\n",
        "\n",
        "    q_table = np.zeros((n_states, n_actions), dtype=np.float32)\n",
        "    returns = np.zeros(episodes, dtype=np.float32)\n",
        "    epsilons = np.zeros(episodes, dtype=np.float32)\n",
        "\n",
        "    for ep in range(episodes):\n",
        "        reset_out = env.reset(seed=seed + ep)\n",
        "        state = toint(resetobs(reset_out))\n",
        "\n",
        "        total_reward = 0.0\n",
        "        for t in range(max_steps):\n",
        "            action = epsilon_greedy_action(q_table, state, epsilon, n_actions)\n",
        "            step_out = env.step(action)\n",
        "            next_state, reward, done = stepunpack(step_out)\n",
        "\n",
        "            s = toint(state)\n",
        "            s_next = toint(next_state)\n",
        "\n",
        "            # Q-learning update\n",
        "            best_next = float(np.max(q_table[s_next]))\n",
        "            td_target = float(reward) + (0.0 if done else gamma * best_next)\n",
        "            td_error = td_target - q_table[s, action]\n",
        "            q_table[s, action] += alpha * td_error\n",
        "\n",
        "            state = s_next\n",
        "            total_reward += float(reward)\n",
        "            if done:\n",
        "                break\n",
        "\n",
        "        returns[ep] = total_reward\n",
        "        epsilons[ep] = epsilon\n",
        "        epsilon = max(epsilon_min, epsilon * epsilon_decay)\n",
        "\n",
        "        if verbose and (ep + 1) % max(1, episodes // 10) == 0:\n",
        "            window = 200 if episodes >= 200 else max(1, episodes // 5)\n",
        "            avg_recent = float(np.mean(returns[max(0, ep - window + 1):ep + 1]))\n",
        "            print(f\"[{ep+1:6d}/{episodes}] avg_return(last {window}) = {avg_recent:.3f}, epsilon={epsilon:.3f}\")\n",
        "\n",
        "    env.close()\n",
        "    return q_table, returns, epsilons\n",
        "\n",
        "\n",
        "def evaluate_policy(env_id: str,\n",
        "                    q_table: np.ndarray,\n",
        "                    episodes: int = 100,\n",
        "                    max_steps: int = 200,\n",
        "                    is_slippery: bool | None = None,\n",
        "                    seed: int = 9999):\n",
        "    env = make_env(env_id, is_slippery=is_slippery, render_mode=None, seed=seed)\n",
        "\n",
        "    total_rewards = []\n",
        "    steps_taken = []\n",
        "\n",
        "    for ep in range(episodes):\n",
        "        reset_out = env.reset(seed=seed + ep)\n",
        "        state = toint(resetobs(reset_out))\n",
        "        ep_reward = 0.0\n",
        "        for t in range(max_steps):\n",
        "            action = greedyaction(q_table[state])\n",
        "            step_out = env.step(action)\n",
        "            next_state, reward, done = stepunpack(step_out)\n",
        "\n",
        "            ep_reward += float(reward)\n",
        "            state = toint(next_state)\n",
        "            if done:\n",
        "                steps_taken.append(t + 1)\n",
        "                break\n",
        "        else:\n",
        "            steps_taken.append(max_steps)\n",
        "        total_rewards.append(ep_reward)\n",
        "\n",
        "    env.close()\n",
        "    return float(np.mean(total_rewards)), float(np.mean(steps_taken))\n",
        "\n",
        "\n",
        "def main():\n",
        "    parser = argparse.ArgumentParser(description=\"Tabular Q-Learning for discrete Gym environments\")\n",
        "    parser.add_argument('--env', type=str, default='FrozenLake-v1')\n",
        "    parser.add_argument('--episodes', type=int, default=20000)\n",
        "    parser.add_argument('--max_steps', type=int, default=200)\n",
        "    parser.add_argument('--alpha', type=float, default=0.8)\n",
        "    parser.add_argument('--gamma', type=float, default=0.95)\n",
        "    parser.add_argument('--epsilon', type=float, default=1.0)\n",
        "    parser.add_argument('--epsilon_min', type=float, default=0.01)\n",
        "    parser.add_argument('--epsilon_decay', type=float, default=0.9995)\n",
        "    parser.add_argument('--is_slippery', type=int, default=None, choices=[0, 1])\n",
        "    parser.add_argument('--seed', type=int, default=42)\n",
        "    parser.add_argument('--no_verbose', action='store_true')\n",
        "\n",
        "    # FIX for Jupyter/Colab extra arguments\n",
        "    args, _ = parser.parse_known_args()\n",
        "\n",
        "    print(f\"Using backend: {GYMN}\")\n",
        "    print(f\"Training {args.env} | episodes={args.episodes}, alpha={args.alpha}, gamma={args.gamma}\")\n",
        "\n",
        "    q_table, returns, eps = train_q_learning(env_id=args.env,\n",
        "                                             episodes=args.episodes,\n",
        "                                             max_steps=args.max_steps,\n",
        "                                             alpha=args.alpha,\n",
        "                                             gamma=args.gamma,\n",
        "                                             epsilon=args.epsilon,\n",
        "                                             epsilon_min=args.epsilon_min,\n",
        "                                             epsilon_decay=args.epsilon_decay,\n",
        "                                             is_slippery=(None if args.is_slippery is None else bool(args.is_slippery)),\n",
        "                                             seed=args.seed,\n",
        "                                             verbose=not args.no_verbose)\n",
        "\n",
        "    avg_reward, avg_steps = evaluate_policy(env_id=args.env,\n",
        "                                            q_table=q_table,\n",
        "                                            max_steps=args.max_steps,\n",
        "                                            is_slippery=(None if args.is_slippery is None else bool(args.is_slippery)))\n",
        "\n",
        "    print(\"\\nEvaluation (greedy policy):\")\n",
        "    print(f\"Average reward over 100 episodes: {avg_reward:.3f}\")\n",
        "    print(f\"Average steps to termination: {avg_steps:.1f}\")\n",
        "\n",
        "    if args.env.startswith(\"FrozenLake\"):\n",
        "        print(\"\\nTips for FrozenLake:\")\n",
        "        print(\"- Slippery=True → Use >=20k episodes and slower epsilon decay (0.9997).\")\n",
        "        print(\"- Slippery=False → 3k–10k episodes often suffice.\")\n",
        "    elif args.env.startswith(\"Taxi\"):\n",
        "        print(\"\\nTips for Taxi:\")\n",
        "        print(\"- Learns faster: ~5k–10k episodes work well.\")\n",
        "        print(\"- Use slightly faster epsilon decay like 0.995.\")\n",
        "\n",
        "    np.save(\"q_table.npy\", q_table)\n",
        "    np.save(\"returns.npy\", returns)\n",
        "    np.save(\"epsilons.npy\", eps)\n",
        "    print(\"\\nSaved: q_table.npy, returns.npy, epsilons.npy\")\n",
        "\n",
        "\n",
        "if __name__ == '__main__':\n",
        "    try:\n",
        "        main()\n",
        "    except KeyboardInterrupt:\n",
        "        print(\"Interrupted by user.\")\n"
      ]
    },
    {
      "cell_type": "code",
      "source": [],
      "metadata": {
        "id": "g5W6o1qUCOTn"
      },
      "execution_count": None,
      "outputs": []
    }
  ]
}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'provenance': [],
   'authorship_tag': 'ABX9TyOymC+jmASj24GAG1/9MUXC',
   'include_colab_link': True},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'}},
 'cells': [{'cell_type': 'markdown',
   'metadata': {'id': 'view-in-github', 'colab_type': 'text'},
   'source': ['<a href="https://colab.research.google.com/github/Goutham345/Reinforcement_Learning/blob/main/lab-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>']},
  {'cell_type': 'code',
   'execution_count': 3,
   'metadata': {'colab': {'base_uri': 'https://localhost:8080/'},
    'id': 'i2KehKKYBT4R',
    'outputId': 'b3003792-69de-46d0-f108-a1f8add4ea70'},
   'outputs': [{'output_type': 'stream',
     'name': 'stdout',
     'text': ['Using backend: gymnasium\n',
      'Training FrozenLake-v1 | episodes=20000, alpha=0.8, gamma=0.95\n',
      '[  200